In [5]:
%matplotlib

Using matplotlib backend: MacOSX


In [38]:
from pathlib import Path
import re
from pprint import pprint

import numpy as np
from scipy import signal, ndimage
import matplotlib.pyplot as plt
from matplotlib import pyplot
import pandas as pd

import emd
import eelbrain
import mne
import trftools

## How to get the IF from the envelope, and turn it into the TRF?

1. Load in the envelope
2. Emd sift the envelope
3. Save the IF value
4. Pack the IF value with the sensor
5. Calculate the IF's TRF

In [3]:
## NATIVES ##

STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results")  #Path("~").expanduser() / 'Data' / 'Alice'
PREDICTOR_audio_DIR = DATA_ROOT / 'TRFs_pridictors/audio_predictors'
PREDICTOR_word_DIR = DATA_ROOT / 'TRFs_pridictors/word_predictors'
EEG_DIR = DATA_ROOT / 'EEG_Natives' / 'Alice_natives_ICAed_fif'
Native_SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name)]
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs_Natives'
TRF_DIR.mkdir(exist_ok=True)
print(Native_SUBJECTS)
print(len(Native_SUBJECTS))

['S13_Alice-natives_sfreq-100_raw.fif', 'S14_Alice-natives_sfreq-100_raw.fif', 'S12_Alice-natives_sfreq-100_raw.fif', 'S15_Alice-natives_sfreq-100_raw.fif', 'S11_Alice-natives_sfreq-100_raw.fif', 'S16_Alice-natives_sfreq-100_raw.fif', 'S17_Alice-natives_sfreq-100_raw.fif', 'S18_Alice-natives_sfreq-100_raw.fif', 'S19_Alice-natives_sfreq-100_raw.fif', 'S20_Alice-natives_sfreq-100_raw.fif', 'S21_Alice-natives_sfreq-100_raw.fif', 'S01_Alice-natives_sfreq-100_raw.fif', 'S03_Alice-natives_sfreq-100_raw.fif', 'S04_Alice-natives_sfreq-100_raw.fif', 'S05_Alice-natives_sfreq-100_raw.fif', 'S06_Alice-natives_sfreq-100_raw.fif', 'S08_Alice-natives_sfreq-100_raw.fif', 'S10_Alice-natives_sfreq-100_raw.fif', 'S22_Alice-natives_sfreq-100_raw.fif', 'S25_Alice-natives_sfreq-100_raw.fif', 'S26_Alice-natives_sfreq-100_raw.fif', 'S34_Alice-natives_sfreq-100_raw.fif', 'S35_Alice-natives_sfreq-100_raw.fif', 'S36_Alice-natives_sfreq-100_raw.fif', 'S37_Alice-natives_sfreq-100_raw.fif', 'S38_Alice-natives_sfreq

In [46]:
# Load the broad-band envelope from the gammatone pickle files
envelope = [eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]  # Load in the data
"""# Code Explanation
envelopeL = []
for stimulus in STIMULI:
    envelope = eelbrain.load.unpickle(PREDICTOR_audio_DIR / f'{stimulus}~gammatone-1.pickle')
    print(envelope)
    envelopeL.append(envelope)
"""
# To down sample into 100 Hz
envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
#print(envelope)
""" # Code Explanation
envelopeL_2 = []
for x in envelope:
    envelope = x.bin(0.01, dim='time', label='start')
    envelopeL_2.append(envelope)
"""
# To covert it into the NDVar structures
envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
""" # Code Explanation
envelopeL_3 = []
for x in envelope:
    envelope = trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope')
    envelopeL_3.append(envelope)
print(envelopeL_3)
"""

print(envelope)
#dir(envelope[1])
#print(envelope[1].get_data())

"""
# Plot every envelope (Don't know why it all overlay into one figs)
for i in range(0, 2):
    p = None
    print(envelope[i].get_data())
    p = plt.plot(envelope[i].get_data())
"""

[<NDVar 'envelope': 5862 time>, <NDVar 'envelope': 6193 time>, <NDVar 'envelope': 6434 time>, <NDVar 'envelope': 7107 time>, <NDVar 'envelope': 6736 time>, <NDVar 'envelope': 6487 time>, <NDVar 'envelope': 6398 time>, <NDVar 'envelope': 5839 time>, <NDVar 'envelope': 5831 time>, <NDVar 'envelope': 6235 time>, <NDVar 'envelope': 5725 time>, <NDVar 'envelope': 4807 time>]


"\n# Plot every envelope (Don't know why it all overlay into one figs)\nfor i in range(0, 2):\n    p = None\n    print(envelope[i].get_data())\n    p = plt.plot(envelope[i].get_data())\n"

In [64]:
sample_rate = 100
"""  # For IMF & IF testing commands
imf, mask_freqs = emd.sift.mask_sift(envelope[1].get_data(), ret_mask_freq=True, max_imfs=6)  
# delete >>, mask_freqs=30/sample_rate
# xy = envelope data 
#print(mask_freqs * sample_rate)
#emd.plotting.plot_imfs(imf)
IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'nht')

pprint(IF[1:])
print(IF.shape)
#print(IF)
"""
#IF_ndarray = np.zeros(shape=[73665, 6])
for i in range(0, 12):
    imf, mask_freqs = emd.sift.mask_sift(envelope[i].get_data(), ret_mask_freq=True, max_imfs=6)  
    # delete >>, mask_freqs=30/sample_rate   # xy = envelope data 
    print(mask_freqs * sample_rate)
    #emd.plotting.plot_imfs(imf)
    IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') #get the IF
    print(IF)
    
    np.append(IF_ndarray, IF)
    print(IF_ndarray)
#print(IF_ndarray)

"""
dataDICT = pd.DataFrame(IF, columns = ['IMF1','IMF2','IMF3', 'IMF4','IMF5','IMF6'])
file_name = 'ALL_IF_TRF_predictor_testing_tables.csv'
save_path = DATA_ROOT/ "TRFs_pridictors" / Path(file_name)
dataDICT.to_csv(save_path, sep = "," ,index = False , header = True, encoding = "UTF-8")
"""
"""
[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]>> = 每個IMF最多的頻率
# 每個ＩＭＦ的順時頻率都做一個TRF 去跑統計
4hz>> close to the syllables, 18 hz >> close to phoneme speed, but normally we consider syllables as speed
>> use the third IMF >> 
# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') get the IF
"""

[21.51142955 10.75571477  5.37785739  2.68892869  1.34446435  0.67223217]
[[ 9.24435029e+00 -2.33195033e-16  4.05918816e+00  4.66435550e+00
   5.32581468e+00  4.67941091e+00]
 [ 9.24435029e+00  6.22015288e-16  4.05918816e+00  4.66435550e+00
   5.32581468e+00  4.67941091e+00]
 [ 9.94324013e+00  3.06814026e+00  7.05943204e+00  2.84101700e+00
   3.00245326e+00  2.29723135e+00]
 ...
 [-1.87900138e-09  1.22679645e-12 -2.53624083e-02  9.38037446e-03
  -1.85458705e-01 -1.61771444e+00]
 [-1.29999907e-09 -2.66166915e-05 -2.56061170e-02  3.48324789e-13
  -3.12581256e-01 -1.85297888e+00]
 [-1.31819302e-09 -2.66166999e-05 -2.56061170e-02 -7.41447127e-14
  -3.12581256e-01 -1.85297888e+00]]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]
[[ 1.86117052e+00  3.59562465e+00  6.46649204e+00  6.60651883e+00
  -1.98333328e-15  2.59778575e+00]
 [ 1.86

"\n[18.86807686  9.43403843  4.71701922  2.35850961  1.1792548   0.5896274 ]>> = 每個IMF最多的頻率\n# 每個ＩＭＦ的順時頻率都做一個TRF 去跑統計\n4hz>> close to the syllables, 18 hz >> close to phoneme speed, but normally we consider syllables as speed\n>> use the third IMF >> \n# IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'hilbert') get the IF\n"

In [41]:
# To save the IF into a csv file
dataDICT = pd.DataFrame(IF, columns = ['IMF1','IMF2','IMF3', 'IMF4','IMF5','IMF6'])
#data_path = Path("/Volumes/Neurolang_1/Master Program/New_Thesis_topic/Experiments_Results/TRFs_pridictors")
file_name = 'IF_TRF_predictor_testing_tables.csv'
save_path = DATA_ROOT/ "TRFs_pridictors" / Path(file_name)
dataDICT.to_csv(save_path, sep = "," ,index = False , header = True, encoding = "UTF-8")